# Health Survey Data Analysis
Olalekan Hammed


## Executive Summary

TODO: write an introduction using Markdown, explaining your high-level project goals and findings

## The Data

TODO: write a summary of the data source as well as an explanation of the `PHYSHLTH` and `RENTHOM1` columns using Markdown

In [28]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import scipy.stats as stats


file_path = "https://raw.githubusercontent.com/learn-co-curriculum/da-phase3-project-enterprise/main/brfss_ny.csv"
df = pd.read_csv(file_path)
df.head()

,_STATE,FMONTH,IDATE,IMONTH,IDAY,IYEAR,DISPCODE,SEQNO,_PSU,CTELENM1,...,_RFPSA23,_CLNSCPY,_SGMSCPY,_SGMS10Y,_RFBLDS4,_STOLDNA,_VIRCOLN,_SBONTIM,_CRCREC1,_AIDTST4
0,36,11,11172020,11,17,2020,1200,2020000001,2020000001,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
1,36,1,1182020,1,18,2020,1200,2020000002,2020000002,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0
2,36,4,5042020,5,4,2020,1200,2020000003,2020000003,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0
3,36,1,1222020,1,22,2020,1200,2020000004,2020000004,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0
4,36,3,3102020,3,10,2020,1200,2020000005,2020000005,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0


## Data Preparation

TODO: write an explanation of the data preparation/cleaning steps you are about to perform using Markdown

In [29]:
# Extract PHYSHLTH & RENTHOM1 into a dafa frame
df_phrh = df[['PHYSHLTH','RENTHOM1']]
df_phrh.head()

,PHYSHLTH,RENTHOM1
0,88.0,2.0
1,14.0,1.0
2,88.0,2.0
3,5.0,3.0
4,88.0,2.0


In [30]:
#get unique values for PHYSHLTH & RENTHOM1
print("PHYSHLTH unique values")
print(df_phrh.PHYSHLTH.unique())
print("RENTHOM1 unique values")
print(df_phrh.RENTHOM1.unique())

PHYSHLTH unique values
[88. 14.  5.  2.  3. 10. 30. 99. 20.  1.  7. 15. 77. 25.  8.  4. 21.  6.
 29.  9. 28. 18. 24. 17. 12. 13. 16. 26. 27. 23. 19. 11. 22.]
RENTHOM1 unique values
[2. 1. 3. 9. 7.]


In [32]:
#dropping records with values of RENTHOM1 other than 1 or 2.
df_phrh.drop(df_phrh.index[df_phrh['RENTHOM1'] > 2], inplace=True)
print(df_phrh)

       PHYSHLTH  RENTHOM1
0          88.0       2.0
1          14.0       1.0
2          88.0       2.0
4          88.0       2.0
5           2.0       2.0
...         ...       ...
14763      88.0       2.0
14764      88.0       2.0
14765      88.0       2.0
14766       2.0       2.0
14767      88.0       2.0

[13957 rows x 2 columns]


/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4913: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [33]:
#check RENTHOM1 unique values
print(df_phrh.RENTHOM1.unique())

[2. 1.]


In [34]:
# Dropping records with 77, 99 in PHYSHLTH column
df_phrh.drop(df_phrh.index[(df_phrh['PHYSHLTH'] == 77 ) | (df_phrh['PHYSHLTH'] == 99)], inplace=True)
print(df_phrh)

       PHYSHLTH  RENTHOM1
0          88.0       2.0
1          14.0       1.0
2          88.0       2.0
4          88.0       2.0
5           2.0       2.0
...         ...       ...
14763      88.0       2.0
14764      88.0       2.0
14765      88.0       2.0
14766       2.0       2.0
14767      88.0       2.0

[13659 rows x 2 columns]


/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4913: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [35]:
#check PHYSHLTH unique values
print(df_phrh.PHYSHLTH.unique())

[88. 14.  2.  3. 10. 30.  5. 20.  1.  7. 15. 25.  8.  4.  6. 29.  9. 21.
 28. 18. 24. 17. 12. 13. 16. 27. 26. 23. 19. 11. 22.]


In [36]:
# Converting all instances of 88 to 0
df_phrh["PHYSHLTH"].replace({88: 0}, inplace=True)
print(df_phrh)

       PHYSHLTH  RENTHOM1
0           0.0       2.0
1          14.0       1.0
2           0.0       2.0
4           0.0       2.0
5           2.0       2.0
...         ...       ...
14763       0.0       2.0
14764       0.0       2.0
14765       0.0       2.0
14766       2.0       2.0
14767       0.0       2.0

[13659 rows x 2 columns]


/usr/local/lib/python3.7/dist-packages/pandas/core/generic.py:6619: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return self._update_inplace(result)


In [37]:
#check PHYSHLTH unique values
print(df_phrh.PHYSHLTH.unique())

[ 0. 14.  2.  3. 10. 30.  5. 20.  1.  7. 15. 25.  8.  4.  6. 29.  9. 21.
 28. 18. 24. 17. 12. 13. 16. 27. 26. 23. 19. 11. 22.]


## Confidence Interval for Mean of `PHYSHLTH`

Note
- The dataset in this survey represent a sample of the more than 20 million New York State rsidents

In [39]:
# the mean of PHYSHLTH
sample_mean = df_phrh['PHYSHLTH'].mean()

# the standard deviation of PHYSHLTH
sample_sd = df_phrh['PHYSHLTH'].std()

# the standard error of PHYSHLTH
sample_se = stats.sem(df_phrh['PHYSHLTH'])

#sample size
sample_size = len(df_phrh['PHYSHLTH'])

#output
print('the sample mean = ', sample_mean)
print('the sample standard deviation = ', sample_sd)
print('the sample standard error = ', sample_se)
print('the sample size = ', sample_size)

the sample mean =  3.064646020938575
the sample standard deviation =  7.58132516485075
the sample standard error =  0.06486877124338632
the sample size =  13659


In [40]:
# calculate 95% confidence interval  using stats.t.interval methods
stats.t.interval(.95, loc=sample_mean, scale=sample_se, df=sample_size-1)

(2.9374942974789744, 3.1917977443981758)

Interpretation
- Given our sample of 13,659, we are 95% confiden the rue mean of the population i.e the number of days of poor physical health New Yorkers typically have in 2020-2021 ranges between 2.94 and 3.19 days

## Differences in `PHYSHLTH` Based on `RENTHOM1`

### How Does `PHYSHLTH` Differ Based on `RENTHOM1`?

TODO: write an explanation of what it means to split the data based on `RENTHOM1` in Markdown

In [41]:
# TODO: write code to separate the records based on the value of RENTHOM1

In [42]:
# TODO: write code to produce one or more plots that show the value of
# PHYSHLTH based on the value of RENTHOM1

TODO: write an explanation of the plot(s) above using Markdown

### Is the Difference Statistically Significant?

TODO: describe the null and alternative hypotheses in Markdown, including whether this is a one-tailed or two-tailed test

In [43]:
# TODO: write code to calculate the test statistic and p-value for a t-test

TODO: interpret the result of the above statistical test using Markdown. Can we reject the null hypothesis at an alpha of 0.05?

## Next Steps

TODO: describe what feature from the CDC codebook that you would propose investigating next, using Markdown